###Algoritmos ICA-PE
*   Algoritmo entrenado y confiable KNN (K-Nearest Neighbors) que demuestre el
accuracy del código de clasificación ICA-PE inicial.

* Algoritmo entrenado y confiable Random Forest que demuestre el accuracy del
código de clasificación ICA-PE inicial.
* Algoritmo entrenado y confiable SVM (Support Vector Machine) que demuestre el
accuracy del código de clasificación ICA-PE inicial.
* Algoritmo para el cálculo del IC-PE basado en las fórmulas de la documentación
dadas por el CLIENTE.



#Inicio del proyecto: 21 de julio del 2024.
* Primera revisión: 27 de julio del 2024.
* Segunda revisión: 3 de agosto del 2024.
* Tercera revisión: 10 de agosto del 2024.
* Entrega final: 17 de agosto del 2024.


#Importaciones

In [ ]:
#Importaciones
import os
import pandas as pd
from typing import List, Dict, Union, Callable, TypeVar, Type
import math
from abc import ABC, abstractmethod
import numpy as np
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
import joblib
import matplotlib.pyplot as plt
import random
from collections import Counter
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# %pip install openpyxl

# Estados iniciales

## Estados iniciales Clase Eca

In [ ]:
Operation = Callable[[float, float], bool]

EcaType = TypeVar('EcaType', bound='Eca')

compare_operations : Dict[str,Operation ] = {
  'greater_than': lambda x, y: x > y,
  'less_than': lambda x, y: x < y,
  'equal_to': lambda x, y: x == y,
  'greater_or_equal': lambda x, y: x >= y,
  'less_or_equal': lambda x, y: x <= y
}

## Estados iniciales Clase Category1A2

In [ ]:
catgory1A2Propertynumber = 13

category1A2Nominals: Dict[str, Union[float, int]] = {
      'O': 5.0,
      'DBO': 5.0,
      'AS_': 0.01,
      'CD': 0.005,
      'CU': 2.0,
      'CR': 0.05,
      'FE': 1.0,
      'MN': 0.4,
      'PB': 0.05,
      'HG': 0.002,
      'ZN': 5.0,
      'MIN_PH': 5.5,
      'MAX_PH': 9.0,
      'MAX_NMP_COLIFORMES': 2000
}

category1A2HeadersA: List[Dict[str,str]] = [
    {
      'key' : '_dbo',
      'label' : 'dbo'
    },
    {
      'key' : '_mn',
      'label' : 'mn'
    },
    {
      'key' : '_zn',
      'label' : 'zn'
    },
    {
      'key' : '_coliformes',
      'label' : 'coliformes'
    },
]

category1A2HeadersB: List[Dict[str,str]] = [
    {
      'key' : '_o',
      'label' : 'o'
    },
    {
      'key' : '_fe',
      'label' : 'fe'
    },
    {
      'key' : '_ph',
      'label' : 'ph'
    },
    {
      'key' : '_cu',
      'label' : 'cu'
    },
]

category1A2HeadersC: List[Dict[str,str]] = [
    {
      'key' : '_as_',
      'label' : 'as'
    },
    {
      'key' : '_cd',
      'label' : 'cd'
    },
    {
      'key' : '_pb',
      'label' : 'pb'
    },
]

category1A2HeadersD: List[Dict[str,str]] = [
    {
      'key' : '_cr',
      'label' : 'cr'
    },
    {
      'key' : '_hg',
      'label' : 'hg'
    }
]

category1A2Probes: List[Dict[str, Union[float, int]]] = [
   {
     'ATRIBUTE' : '_o',
     'NOMINAL' : 'O',
     'OPERATION': 'greater_or_equal',
   },
   {
     'ATRIBUTE' : '_dbo',
     'NOMINAL' : 'DBO',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_as_',
     'NOMINAL' : 'AS_',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_cd',
     'NOMINAL' : 'CD',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_cu',
     'NOMINAL' : 'CU',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_cr',
     'NOMINAL' : 'CR',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_fe',
     'NOMINAL' : 'FE',
     'OPERATION': 'less_than'
   },
   {
     'ATRIBUTE' : '_mn',
     'NOMINAL' : 'MN',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_pb',
     'NOMINAL' : 'PB',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_hg',
     'NOMINAL' : 'HG',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_zn',
     'NOMINAL' : 'ZN',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_ph',
     'NOMINAL' : 'MIN_PH',
     'OPERATION': 'greater_than',
   },
   {
     'ATRIBUTE' : '_ph',
     'NOMINAL' : 'MAX_PH',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_coliformes',
     'NOMINAL' : 'MAX_NMP_COLIFORMES',
     'OPERATION': 'less_than',
   }
 ]

##Estados iniciales Clase Category3

In [ ]:
Category3Type = TypeVar('Category3Type', bound='Category3')

category3Propertynumber = 17

restrictedIrrigationNominals: Dict[str, Union[float, int]] = {
      'CLORUROS': 500,
      'CONDUCTIVIDAD': 2500,
      'DBO': 15,
      'O': 4.0,
      'AL':5.0,
      'AS_': 0.1,
      'B': 1.0,
      'CD': 0.01,
      'CU': 0.2,
      'FE': 5.0,
      'MN': 0.2,
      'HG': 0.001,
      'PB': 0.05,
      'ZN': 2.0,
      'MIN_PH': 6.5,
      'MAX_PH': 8.5,
      'MAX_NMP_COLIFORMES': 2000,
      'MAX_HUEVOS_DE_EMINTOS' : 1
}

noRestrictedIrrigationNominals: Dict[str, Union[float, int]] = {
      'CLORUROS': 500,
      'CONDUCTIVIDAD': 2500,
      'DBO': 15,
      'O': 4.0,
      'AL':5.0,
      'AS_': 0.1,
      'B': 1.0,
      'CD': 0.01,
      'CU': 0.2,
      'FE': 5.0,
      'MN': 0.2,
      'HG': 0.001,
      'PB': 0.05,
      'ZN': 2.0,
      'MIN_PH': 6.5,
      'MAX_PH': 8.5,
      'MAX_NMP_COLIFORMES': 1000,
      'MAX_HUEVOS_DE_EMINTOS' : 1
}

animalDrinkingWaterNominals: Dict[str, Union[float, int]] = {
      'CONDUCTIVIDAD': 2500,
      'DBO': 15,
      'O': 5.0,
      'AL':5.0,
      'AS_': 0.2,
      'B': 5.0,
      'CD': 0.05,
      'CU': 0.05,
      'MN': 0.2,
      'HG': 0.01,
      'PB': 0.05,
      'ZN': 24.0,
      'MIN_PH': 6.5,
      'MAX_PH': 8.4,
      'MAX_NMP_COLIFORMES': 1000
}


category3HeadersA: List[Dict[str,str]] = [
    {
      'key': '_conductividad',
      'label':  'conductividad'
    },
    {
      'key': '_coliformes',
      'label':  'coliformes'
    },
    {
      'key': '_al',
      'label':  'al'
    },
]

category3HeadersB: List[Dict[str,str]] = [
    # {
    #   'key': '_cloruros',
    #   'label':  'cloruros'
    # },
    {
      'key': '_dbo',
      'label':  'dbo'
    },
    # {
    #   'key': '_fe',
    #   'label':  'fe'
    # },
    {
      'key': '_mn',
      'label':  'mn'
    },
    {
      'key': '_zn',
      'label':  'zn'
    },
]

category3HeadersC: List[Dict[str,str]] = [
    {
      'key': '_ph',
      'label':  'ph'
    },
    {
      'key': '_o',
      'label':  'o'
    },
    {
      'key': '_cu',
      'label':  'cu'
    }
]

category3HeadersD: List[Dict[str,str]] = [
    {
      'key': '_as_',
      'label':  'as'
    },
    {
      'key': '_b',
      'label':  'b'
    },
    {
      'key': '_cd',
      'label':  'cd'
    },
    {
      'key': '_hg',
      'label':  'hg'
    },
    {
      'key': '_pb',
      'label':  'pb'
    },
    # {
    #   'key': '_huevos_de_hemintos',
    #   'label':  'huevos_de_hemintos'
    # }
]

category3Probes: List[Dict[str, Union[float, int]]] = [
   {
     'ATRIBUTE' : '_cloruros',
     'NOMINAL' : 'CLORUROS',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_conductividad',
     'NOMINAL' : 'CONDUCTIVIDAD',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_dbo',
     'NOMINAL' : 'DBO',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_o',
     'NOMINAL' : 'O',
     'OPERATION': 'greater_or_equal',
   },
   {
     'ATRIBUTE' : '_al',
     'NOMINAL' : 'AL',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_as_',
     'NOMINAL' : 'AS_',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_b',
     'NOMINAL' : 'B',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_cd',
     'NOMINAL' : 'CD',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_cu',
     'NOMINAL' : 'CU',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_fe',
     'NOMINAL' : 'FE',
     'OPERATION': 'less_than'
   },
   {
     'ATRIBUTE' : '_mn',
     'NOMINAL' : 'MN',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_hg',
     'NOMINAL' : 'HG',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_pb',
     'NOMINAL' : 'PB',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_zn',
     'NOMINAL' : 'ZN',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_ph',
     'NOMINAL' : 'MIN_PH',
     'OPERATION': 'greater_than',
   },
   {
     'ATRIBUTE' : '_ph',
     'NOMINAL' : 'MAX_PH',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_coliformes',
     'NOMINAL' : 'MAX_NMP_COLIFORMES',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_huevos_de_hemintos',
     'NOMINAL' : 'MAX_HUEVOS_DE_EMINTOS',
     'OPERATION': 'less_than',
   }
 ]

animalDrinkingWaterProbes: List[Dict[str, Union[float, int]]] = [
   {
     'ATRIBUTE' : '_conductividad',
     'NOMINAL' : 'CONDUCTIVIDAD',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_dbo',
     'NOMINAL' : 'DBO',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_o',
     'NOMINAL' : 'O',
     'OPERATION': 'greater_or_equal',
   },
   {
     'ATRIBUTE' : '_al',
     'NOMINAL' : 'AL',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_as_',
     'NOMINAL' : 'AS_',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_b',
     'NOMINAL' : 'B',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_cd',
     'NOMINAL' : 'CD',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_cu',
     'NOMINAL' : 'CU',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_mn',
     'NOMINAL' : 'MN',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_hg',
     'NOMINAL' : 'HG',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_pb',
     'NOMINAL' : 'PB',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_zn',
     'NOMINAL' : 'ZN',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_ph',
     'NOMINAL' : 'MIN_PH',
     'OPERATION': 'greater_than',
   },
   {
     'ATRIBUTE' : '_ph',
     'NOMINAL' : 'MAX_PH',
     'OPERATION': 'less_than',
   },
   {
     'ATRIBUTE' : '_coliformes',
     'NOMINAL' : 'MAX_NMP_COLIFORMES',
     'OPERATION': 'less_than',
   }
 ]


# Clases

# Clase KNN

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    @staticmethod
    def EUCLIDIANA(x, y):
        return np.sqrt(np.sum((x - y) ** 2))

    def aprendizaje(self, X, C):
        self.X = X  # matriz de vectores de características
        self.c = C  # clases asociadas a cada vector x(n)
        self.n_muestras = X.shape[1]  # cantidad de muestras

    def clasificacion(self, Y):
        clases = []
        for i in range(Y.shape[1]):  # por cada vector y(n) a clasificar
            distancias = np.empty(self.n_muestras)
            for n in range(self.n_muestras):  # por cada vector x(n) de características
                distancias[n] = self.EUCLIDIANA(self.X[:, n], Y[:, i])

            # distancias más cercanas
            k_distancias = np.argsort(distancias)
            # identificar las k distancias - clases
            k_etiqueta = self.c[k_distancias[:self.k]]
            # votación
            c = Counter(k_etiqueta).most_common(1)  # (5,0)
            clases.append(c[0][0])  # almacenamos la clase asignada al vector y(n)
        return clases


## Clase Eca

In [ ]:
class Eca(ABC):

  """
    Representa los valores a conciderar para el cálculo del ICA para agua Categoria 1-A2

    Atributos:
    - _o : Oxigeno disuelto (valor mínimo) mg/L (float),
    - _dbo : Demanda Bioquímica de Oxigeno (DBO5) mg/L (float),
    - _as_ : Arsénico mg/L (float),
    - _cd : Cadmnio mg/L (float),
    - _cu : Cobre mg/L (float),
    - _cr : Cromo total mg/L (float),
    - _fe : Hierro mg/L (float),
    - _mn : Manganeso mg/L (float),
    - _pb : Plomo mg/L (float),
    - _hg : Mercurio mg/L (float),
    - _zn : Zinc mg/L (float),
    - _ph : Potencial de Hidrógeno (pH) Unid. de pH (float),
    - _coliformes : Coliformes Termotolerantes (44,5°C) NMP/ 100 ml (int)

  """

  _o : float = 0
  _dbo : float = 0
  _as_ : float = 0
  _cd : float = 0
  _cu : float = 0
  _cr : float = 0
  _fe : float = 0
  _mn : float = 0
  _pb : float = 0
  _hg : float = 0
  _zn : float = 0
  _ph : float = 0
  _al : float = 0
  _b : float = 0
  _coliformes : float = 0
  _cloruros : float = 0
  _conductividad : float = 0
  _huevos_de_hemintos : float = 0

  propertynumber : int = 0

  NOMINALS: Dict[str, Union[float, int]] = {}
  COMPARES : Dict[str,Operation ] = compare_operations
  PROBES: List[Dict[str, Union[float, int]]] = []
  HEADERSA: List[Dict[str,str]] = []
  HEADERSB: List[Dict[str,str]] = []
  HEADERSC: List[Dict[str,str]] = []
  HEADERSD: List[Dict[str,str]] = []

  @abstractmethod
  def __init__(self,
             _o: float = 0,
             _dbo: float = 0,
             _as_: float = 0,
             _cd: float = 0,
             _cu: float = 0,
             _cr: float = 0,
             _fe: float = 0,
             _mn: float = 0,
             _pb: float = 0,
             _hg: float = 0,
             _zn: float = 0,
             _ph: float = 0,
             _al: float = 0,
             _b: float = 0,
             _coliformes: float = 0,
             _cloruros: float = 0,
             _conductividad: float = 0,
             _huevos_de_hemintos: float = 0):
    self._o = _o
    self._dbo = _dbo
    self._as_ = _as_
    self._cd = _cd
    self._cu = _cu
    self._cr = _cr
    self._fe = _fe
    self._mn = _mn
    self._pb = _pb
    self._hg = _hg
    self._zn = _zn
    self._ph = _ph
    self._al = _al
    self._b = _b
    self._coliformes = _coliformes
    self._cloruros = _cloruros
    self._conductividad = _conductividad
    self._huevos_de_hemintos = _huevos_de_hemintos

  # Getters
  @property
  def get_o(self) -> float:
      return self._o

  @property
  def get_dbo(self) -> float:
      return self._dbo

  @property
  def get_as_(self) -> float:
      return self._as_

  @property
  def get_cd(self) -> float:
      return self._cd

  @property
  def get_cu(self) -> float:
      return self._cu

  @property
  def get_cr(self) -> float:
      return self._cr

  @property
  def get_fe(self) -> float:
      return self._fe

  @property
  def get_mn(self) -> float:
      return self._mn

  @property
  def get_pb(self) -> float:
      return self._pb

  @property
  def get_hg(self) -> float:
      return self._hg

  @property
  def get_zn(self) -> float:
      return self._zn

  @property
  def get_ph(self) -> float:
      return self._ph

  @property
  def get_coliformes(self) -> float:
      return self._coliformes

  @property
  def get_al(self) -> float:
    return self._al

  @property
  def get_b(self) -> float:
    return self._b

  @property
  def get_cloruros(self) -> float:
    return self._cloruros

  @property
  def get_conductividad(self) -> float:
    return self._conductividad

  @property
  def get_huevos_de_hemintos(self) -> float:
    return self._huevos_de_hemintos

  #-----------------------------------------------------------------------------
  @classmethod
  @abstractmethod
  def createFromDF(cls: Type[EcaType] ,df : pd.DataFrame) -> List[EcaType] :
    pass

  #-----------------------------------------------------------------------------
  @classmethod
  def createFromExcel(cls: Type[EcaType]) -> List[EcaType] :
    """
    Crea una lista de instancias de  a partir de un archivo de Excel.

    Lee los datos de un archivo de Excel utilizando el método `__read_doc`, lo convierte en un DataFrame,
    y crea instancias de  para cada fila en el DataFrame.

    Returns:
        List[]: Una lista de objetos  creados a partir de los datos del Excel.

    Raises:
        KeyError: Si faltan columnas esperadas en los datos del Excel.
        Exception: Para cualquier otro error imprevisto durante el procesamiento.
    """
    #Se lee el archivo y se extrae la data
    data = cls.read_doc()

    #Se instancia en un data frame
    df = pd.DataFrame(data)

    #Se crea el array a partir del dataframe
    categories = cls.createFromDF(df)

    return categories

  #-----------------------------------------------------------------------------
  @classmethod
  def read_doc(cls, carpeta : str = './template' ) -> pd.DataFrame:
    """
    Lee datos de un archivo de Excel ubicado en una carpeta específica.

    Define la ruta del archivo de Excel y, si el archivo existe, lo lee utilizando `pd.read_excel`
    y devuelve el contenido como un DataFrame. Si el archivo no se encuentra, imprime un mensaje de error.

    Returns:
        pd.DataFrame: Los datos del archivo de Excel leídos como un DataFrame.

    Prints:
        str: Mensaje de error si el archivo no se encuentra en la carpeta.
    """

    #Se define el dorectorio para guardar el template
    folder_path = f'./content/{carpeta}'

    if not os.path.exists(carpeta):
      os.makedirs(f'{carpeta}')
      print(f"Directorio '{f'{carpeta}'}' creado.")
      print(f'sube tus archivo de entrenamiento a la carpeta {carpeta}')
      return  pd.DataFrame()
    archivosEncarpeta = [
      os.path.join(os.getcwd(), f'{carpeta}', x)
      for x in os.listdir(f'{carpeta}')
      if not x.startswith(".ipynb_checkpoints")  # Filtrar el archivo .ipynb_checkpoints
      ] # genera todas las rutas de los xslx para leerlos
    # print(f'La carpeta POS contiene : {str(len(categoria1paths))} archivos')
    if len(archivosEncarpeta) ==0:
      print(f"Capeta {carpeta} vacía sube tu excel")
      return  pd.DataFrame()
    #Intenta leer el archivo
    print(archivosEncarpeta[0],"----")
    full_path = os.path.join(folder_path, archivosEncarpeta[0])
    print(full_path)
    if os.path.exists(full_path):
        data = pd.read_excel(full_path, engine='openpyxl')
        return data

    print(f'No se ha encontrado el archivo en la carpeta {carpeta}')
    return  pd.DataFrame()

  #-----------------------------------------------------------------------------
  def getOutOfRangeAtributtes(self) -> List[str]:

    outOfRanges = []

    for probe in self.PROBES:

      value = getattr(self, str(probe['ATRIBUTE']))
      operation = self.COMPARES[str(probe['OPERATION'])]
      nominal = self.NOMINALS[str(probe['NOMINAL'])]

      result = operation(float(value),nominal)

      if not result:
        outOfRanges.append(str(probe['ATRIBUTE']))


    return list(set(outOfRanges))

  #-----------------------------------------------------------------------------
  def getOutOfRangeValues(self) -> int:

    outOfRanges = 0

    for probe in self.PROBES:

      value = getattr(self, str(probe['ATRIBUTE']))
      operation = self.COMPARES[str(probe['OPERATION'])]
      nominal = self.NOMINALS[str(probe['NOMINAL'])]

      result = operation(float(value),nominal)
      if not result:
        outOfRanges = outOfRanges + 1

    return outOfRanges

  #-----------------------------------------------------------------------------
  @classmethod
  def calculateF1(cls : Type[EcaType],arr: List[EcaType]) -> float:

    #Se cálculan las
    outOfRangeAttributes = []

    for category in arr:
      outOfRange = category.getOutOfRangeAtributtes()
      outOfRangeAttributes.extend(outOfRange)

    outOfRangeAttributes = list(set(outOfRangeAttributes))

    attributes = cls.propertynumber

    return len(outOfRangeAttributes) / attributes

  #-----------------------------------------------------------------------------
  @classmethod
  def calculateF2(cls : Type[EcaType],arr: List[EcaType]) -> float:

    #Se cálculan las
    outOfRangeValues = 0

    for category in arr:
      outOfRange = category.getOutOfRangeValues()
      outOfRangeValues = outOfRangeValues + outOfRange
    attributes = cls.propertynumber * len(arr)

    return outOfRangeValues / attributes

  #-----------------------------------------------------------------------------

  @classmethod
  def calculateNse(cls : Type[EcaType],arr: List[EcaType]) -> float:

    functionCatalog = {
        'greater_than':'minor',
        'less_than':'major',
        'greater_or_equal':'minor',
        'less_or_equal':'major',
    }

    #Se cálculan las
    outOfRangeSum = 0

    for index, category in enumerate(arr):

      for probe in cls.PROBES:
        value = getattr(category, str(probe['ATRIBUTE']))
        operation = cls.COMPARES[str(probe['OPERATION'])]
        nominal = cls.NOMINALS[str(probe['NOMINAL'])]

        result = operation(float(value),nominal)
        if not result:

          if functionCatalog[str(probe['OPERATION'])] == 'minor':
            outOfRangeSum = outOfRangeSum + ((nominal/value)-1)


          if functionCatalog[str(probe['OPERATION'])] == 'major':
            outOfRangeSum = outOfRangeSum + (( value / nominal)-1)


    attributes = cls.propertynumber * len(arr)

    return outOfRangeSum / attributes


  #-----------------------------------------------------------------------------
  @classmethod
  def calculateF3(cls : Type[EcaType],arr: List[EcaType]) -> float:

    #Se cálculan las suma normalizada de excedentes
    nse = cls.calculateNse(arr)

    return (nse /(nse + 1)) * 100


  #-----------------------------------------------------------------------------
  @classmethod
  def calculateIca(cls : Type[EcaType], arr: List[EcaType]) -> float :
    f1 = cls.calculateF1(arr)
    f2 = cls.calculateF2(arr)
    f3 = cls.calculateF3(arr)
    # print(f"F1={f1}, F2 = {f2}, F3 = {f3}")

    return 100 - (( f1 **2 + f2 **2 + f3**2 )/3)**0.5


  #-----------------------------------------------------------------------------
  @classmethod
  def generar_SVM_multiclass(cls):
    # data_etiquetar=cls.leer_para_etiquetado() #Lista de objetos Category1A2
    # # print(data_etiquetar)
    # if data_etiquetar==None:
    #   return None
    X,labels=cls.ecaListFromRndDataset(1600)
    print("Lista de factores ",X)
    print("Lista de etiquetas ",labels)
    # Codificar etiquetas de clase
    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(labels)

    # Crear un clasificador SVM con la estrategia "one-vs-one"
    clf = svm.SVC(decision_function_shape='ovo')

    # Entrenar el modelo
    clf.fit(X, Y)
    # Exportar el modelo
    if not os.path.exists(f'./{cls.__name__}'):
      os.makedirs(f'./{cls.__name__}')
      print(f"Directorio '{f'./{cls.__name__}'}' creado.")

    modelo="modelo_svm.pkl"
    joblib.dump(clf, f'./{cls.__name__}/{modelo}')
    print(f"Modelo exportado como './{cls.__name__}/{modelo}'")
    exito = cls.esperar_modelo(clase_nombre=f"{cls.__name__}", archivo=f"{modelo}")
    if exito:
        print("Archivo encontrado y listo para su uso.")
    else:
        print("El archivo no fue encontrado dentro del tiempo de espera especificado.")

    # Guardar también el codificador de etiquetas
    etiquetador="label_encoder_svm.pkl"
    joblib.dump(label_encoder, f'./{cls.__name__}/{etiquetador}')
    print(f"Codificador de etiquetas exportado como './{cls.__name__}/{etiquetador}'")
    exito = cls.esperar_modelo(clase_nombre=f"{cls.__name__}", archivo=f"{etiquetador}")
    if exito:
        print("Archivo encontrado y listo para su uso.")
    else:
        print("El archivo no fue encontrado dentro del tiempo de espera especificado.")
    ### NO BORRAR ESTO (SERVIRÁ CUANDO SE HAGAN LAS OTRAS CATEGORIAS Y SE TENGAN MÁS DATOS)
    # # # # # Datos de prueba
    # # # # X_test = [[0.3, 0.1, 50]]  # Ejemplo de características para un nuevo dato

    # # # # # Realizar predicción
    # # # # prediccion = clf.predict(X_test)
    # # # # print("Predicción:", label_encoder.inverse_transform(prediccion))  # Decodificar predicción

    # # # # # Obtener la función de decisión
    # # # # dec = clf.decision_function(X_test)
    # # # # print("Valores de la función de decisión:", dec)

    # # # # # Verificar la forma de la salida de la función de decisión
    # # # # print("Forma de la salida de la función de decisión:", dec.shape)

    # # # # # Cambiar la estrategia a "one-vs-rest"
    # # # # clf.decision_function_shape = "ovr"

    # # # # # Obtener la función de decisión con la nueva estrategia
    # # # # dec = clf.decision_function(X_test)
    # # # # print("Valores de la función de decisión con 'ovr':", dec)
    # # # # print("Forma de la salida de la función de decisión con 'ovr':", dec.shape)
  #-----------------------------------------------------------------------------
  @classmethod
  def usarModeloSVM(cls,x_predecir) -> str:
    # Cargar el modelo
    modelo_cargado = joblib.load(f'./{cls.__name__}/modelo_svm.pkl')

    # Cargar el codificador de etiquetas
    label_encoder_cargado = joblib.load(f'./{cls.__name__}/label_encoder_svm.pkl')

    # Realizar predicción con el modelo cargado
    prediccion_nueva = modelo_cargado.predict(x_predecir)
    # print("Nueva Predicción:", label_encoder_cargado.inverse_transform(prediccion_nueva))
    return label_encoder_cargado.inverse_transform(prediccion_nueva)[0]

  @classmethod
  def leer_para_etiquetado(cls):
    listacategoria1=[]
    if not os.path.exists(f'./{cls.__name__}_excels'):
      os.makedirs(f'./{cls.__name__}_excels')
      print(f"En esta carpeta sube tus excels ./{cls.__name__}_excels")
      return None
    categoria1paths = [
      os.path.join(os.getcwd(), f'./{cls.__name__}_excels', x)
      for x in os.listdir(f'./{cls.__name__}_excels')
      if not x.startswith(".ipynb_checkpoints")  # Filtrar el archivo .ipynb_checkpoints
      ] # genera todas las rutas de los csv para leerlos
    # print(f'La carpeta POS contiene : {str(len(categoria1paths))} archivos')
    if len(categoria1paths) ==0:
      print("Capeta vacía sube tu excel")
      return None
    for ruta in categoria1paths:
      exceltemp=pd.ExcelFile(ruta)
      for i in exceltemp.sheet_names:
          dftemp = pd.read_excel(ruta, sheet_name=i)
          listacategoria1.append(cls.createFromDF(dftemp))

          # display(dftemp)
          # dftemp.plot(kind='scatter', x='as', y='cu', s=32, alpha=.8)
          # plt.gca().spines[['top', 'right',]].set_visible(False)
    return listacategoria1


  #-----------------------------------------------------------------------------
  @classmethod
  def etiquetar_dataFrames(cls,listaobjetos):
    labels=[]
    factoresefe=[]
    for periodos in listaobjetos:
      valorICA=cls.calculateIca(periodos)
      # print(valorICA)
      # print(type(valorICA))
      etiqueta=cls.asignLabel(valorICA)
      labels.append(etiqueta)
      f1 = cls.calculateF1(periodos)
      f2 = cls.calculateF2(periodos)
      f3 = cls.calculateF3(periodos)
      factoresefe.append([f1,f2,f3])
    return factoresefe,labels


  #-----------------------------------------------------------------------------
  @staticmethod
  def asignLabel(ica : float) -> str :

    ica = math.floor(ica)
    if 90<=ica <=100:
      return "Excelente"
    elif 75<=ica <90:
      return "Bueno"
    elif 45<=ica <=74:
      return "Regular"
    elif 30<=ica <=44:
      return "Malo"
    elif 0<ica <=29:
      return "Pésimo"
    else:
      print(ica)
      return "N/A"


  @staticmethod
  def createPlot(
      arr: List['EcaType'],
      headers: List[Dict[str, str]],
      position: int,
      total_plots: int
  ) -> None:
      """
      Crea una gráfica en una posición específica dentro de una cuadrícula de subplots.
      Args:
          arr (List[EcaType]): Lista de objetos con datos para graficar.
          headers (List[Dict[str, str]]): Lista de diccionarios que contienen la clave y etiqueta para cada línea en la gráfica.
          position (int): Posición del subplot en la cuadrícula (1 a total_plots).
          total_plots (int): Número total de subplots en la figura.
      """
      import matplotlib.pyplot as plt
      # Definir la disposición de la cuadrícula de subplots
      rows = cols = int(total_plots ** 0.5)
      if rows * cols < total_plots:
          cols += 1
      # Crear el subplot en la posición especificada
      plt.subplot(rows, cols, position)
      data = {}
      axisX = []
      colors = [
          'dodgerblue', 'salmon', 'limegreen', 'darkorange', 'deepskyblue',
          'purple', 'gold', 'cyan', 'magenta', 'olive',
          'navy', 'teal', 'coral', 'orchid', 'sienna',
          'grey', 'brown', 'pink', 'lightgreen', 'lavender'
      ]
      marks = [
          '*', 'o', '+', 'x', 's', 'd', '^', 'v', '<', '>',
          'p', 'h', 'H', 'D', '|', '_', '.', ',', '1', '2'
      ]
      # Generar el eje X basado en la longitud de arr
      axisX = list(range(1, len(arr) + 1))
      # Preparar los datos para cada línea en la gráfica
      for key, header in enumerate(headers):
          data[header['label']] = [getattr(category, header['key']) for category in arr]
      # Graficar cada línea con estilos diferentes
      for key, header in enumerate(headers):
          plt.plot(
              axisX,
              data[header['label']],
              marker=marks[key % len(marks)],
              linestyle='-',
              color=colors[key % len(colors)],
              label=header['label']
          )
      plt.xlabel('Muestreos')
      plt.ylabel('Mediciones')
      plt.title(f'Gráfica {position}')
      plt.legend()
      plt.grid(True)

  #-----------------------------------------------------------------------------
  @classmethod
  def plotCollection(
      cls: Type[EcaType],
      arr: List[EcaType],
      titles: Dict[str,str]
      ) -> None:

      plt.figure(figsize=(12, 6))  # Adjust the figure size as needed

      cls.createPlot(arr, cls.HEADERSA, position=1,total_plots=4)

      cls.createPlot(arr, cls.HEADERSB, position=2, total_plots=4)

      cls.createPlot(arr, cls.HEADERSC, position=3, total_plots=4)

      cls.createPlot(arr, cls.HEADERSD, position=4, total_plots=4)

      fisrtLine = titles['category']
      secondLine = f"ICA calculado     : {titles['ica']}"
      thirthLine = f"Etiqueta formula : {titles['formula']}"
      fourthLine = f"Etiqueta svm      : {titles['svm']}"
      fifthLine = f"Resultado random forest      : {titles['rndmforest']}"
      sextLine = f"Etiqueta KNN      : {titles['knn']}"

      title = f"{fisrtLine}\n{secondLine}\n{thirthLine}\n{fourthLine}\n{fifthLine}\n{sextLine}"

      plt.suptitle(title, x=0.1, ha='left')  # Ajustar 'x' para mover el título a la izquierda

      plt.tight_layout(rect=[0, 0, 1, 0.90])  # Ajustar el diseño para hacer espacio para el título
      plt.show()

  #-----------------------------------------------------------------------------
  @classmethod
  def randomDfList(cls,  num_rows: int = 2000) -> List[pd.DataFrame] :
    df = cls.read_doc('./entrenamiento')

    if df.empty:
      print("El dataframe está vacio")
      return [pd.DataFrame()]

    # Leer las primeras `num_rows` filas del DataFrame
    df_subset = df.head(num_rows)

    newConfig = {}

    for probe in cls.PROBES:

      operation = cls.COMPARES[str(probe['OPERATION'])]
      nominal = cls.NOMINALS[str(probe['NOMINAL'])]
      column = str(probe['ATRIBUTE']).replace('_','')

      test = df_subset[column]
      newConfig[column] = []

      for item in test:

        randomNumber = random.uniform(0,nominal * 2.5)
        newConfig[column].append(randomNumber if math.isnan(item) or item <=0 else item )

    randomDf = pd.DataFrame(newConfig)

    randomDf.to_csv('/content/randomDf.csv', index=False)

    # Definir el número de filas por DataFrame
    rows_per_df = 4

    # Crear una lista para almacenar los DataFrames
    dataframes_list = []

    # Dividir el DataFrame original en partes más pequeñas
    for start in range(0, len(randomDf), rows_per_df):
        end = start + rows_per_df
        df_part = randomDf.iloc[start:end]
        dataframes_list.append(df_part)

    return dataframes_list


  #-----------------------------------------------------------------------------
  @classmethod
  def cutDataset(cls, num_rows: int = 2000) -> pd.DataFrame:

    # guardar_df_en_hojas_aleatorias(randomDf, 'archivo_aleatorio.xlsx')
    dataframes_list = cls.randomDfList(num_rows)

    ecaList = {
        'f1':[],
        'f2' :[],
        'f3' :[],
        'labels' :[]
    }

    # Imprimir los DataFrames resultantes
    for i, df in enumerate(dataframes_list):
      reads = cls.createFromDF(df)
      ica = cls.calculateIca(reads)
      labels = cls.asignLabel(ica)
      f1,f2,f3=cls.calculateF1(reads),cls.calculateF2(reads),cls.calculateF3(reads)
      ecaList['f1'].append(f1)
      ecaList['f2'].append(f2)
      ecaList['f3'].append(f3)
      ecaList['labels'].append(labels)

    trainDf = pd.DataFrame(ecaList)
    trainDf.to_csv('/content/randomTrain.csv', index=False)

    return trainDf

  #-----------------------------------------------------------------------------
  @classmethod
  def ecaListFromRndDataset(cls, num_rows: int = 2000):
    trainDf = cls.cutDataset(num_rows)

    fList = []
    labels = []

    for index, row in trainDf.iterrows():
      fList.append([row['f1'],row['f2'],row['f3']])
      labels.append(row['labels'])

    return fList, labels

  #-----------------------------------------------------------------------------
  @classmethod
  def clasifyKNN(cls, reads : List[List[float]]) -> str:


      trainDf = cls.cutDataset()

      # Extracción de las características y transposición de la matriz para KNN
      X = trainDf[['f1', 'f2', 'f3']].values.T  # Transpuesta para que coincida con la implementación KNN

      # Extracción de las etiquetas
      C = trainDf['labels'].values

      # Convertir las etiquetas a números si es necesario
      etiquetas_unicas = list(set(C))
      etiquetas_map = {etiqueta: i for i, etiqueta in enumerate(etiquetas_unicas)}
      C = np.array([etiquetas_map[etiqueta] for etiqueta in C])

      # Graficar los datos de entrenamiento
      fig = plt.figure()
      ax = fig.add_subplot(111, projection='3d')

      for etiqueta, color, marker in zip(etiquetas_unicas, ['r', 'g', 'b'], ['o', '^', 's']):
          # Filtra los datos por etiqueta
          idx = np.where(C == etiquetas_map[etiqueta])
          ax.scatter(X[0, idx], X[1, idx], X[2, idx], c=color, marker=marker, label=etiqueta)

      ax.set_xlabel('f1')
      ax.set_ylabel('f2')
      ax.set_zlabel('f3')
      ax.set_title('Datos de Entrenamiento')
      plt.legend()
      plt.show()

      Y = np.array(reads)

      clasificador = KNN(k=10)
      clasificador.aprendizaje(X, C)  # fase de aprendizaje
      clasificar = clasificador.clasificacion(Y)

      # Convertir de vuelta a las etiquetas originales
      clases_etiquetas = [etiquetas_unicas[clase] for clase in clasificar]

      # Retorna las etiquetas clasificadas como una cadena
      return clases_etiquetas[0]

  #-----------------------------------------------------------------------------
  @classmethod
  def generar_randomForest_multiclass(cls):
    # data_etiquetar=cls.leer_para_etiquetado() #Lista de objetos Category1A2
    # # print(data_etiquetar)
    # if data_etiquetar==None:
    #   return None
    X,labels=cls.ecaListFromRndDataset(2000)
    print("Lista de factores ",X)
    print("Lista de etiquetas ",labels)
    # Codificar etiquetas de clase
    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(labels)
    # Codificar las etiquetas de texto a valores numéricos

    # Entrenar el clasificador RandomForest
    clf_rnd = RandomForestClassifier(random_state=42, n_jobs=-1)#-1 indica que usara todo el poder de computo para el modelo
    grid_search= GridSearchCV(clf_rnd,cls.params_grid_random_forest,cv=5,scoring="f1_weighted", return_train_score=True)
    grid_search.fit(X, Y)

    print(grid_search.best_params_)
    print(grid_search.best_estimator_)



    # clf_rnd.fit(X, Y)

    if not os.path.exists(f'./{cls.__name__}'):
      os.makedirs(f'./{cls.__name__}')
      print(f"Directorio '{f'./{cls.__name__}'}' creado.")

    modelo="modelo_rnd_forest.pkl"
    # joblib.dump(clf_rnd, f'./{cls.__name__}/{modelo}')
    joblib.dump(grid_search.best_estimator_, f'./{cls.__name__}/{modelo}')
    print(f"Modelo exportado como './{cls.__name__}/{modelo}'")
    exito = cls.esperar_modelo(clase_nombre=f"{cls.__name__}", archivo=f"{modelo}")
    if exito:
        print("Archivo encontrado y listo para su uso.")
    else:
        print("El archivo no fue encontrado dentro del tiempo de espera especificado.")

    # Guardar también el codificador de etiquetas
    etiquetador="label_encoder_rnd_forest.pkl"
    joblib.dump(label_encoder, f'./{cls.__name__}/{etiquetador}')
    print(f"Codificador de etiquetas exportado como './{cls.__name__}/{etiquetador}'")
    exito = cls.esperar_modelo(clase_nombre=f"{cls.__name__}", archivo=f"{etiquetador}")
    if exito:
        print("Archivo encontrado y listo para su uso.")
    else:
        print("El archivo no fue encontrado dentro del tiempo de espera especificado.")

  #-----------------------------------------------------------------------------
  @staticmethod
  def esperar_modelo(clase_nombre, archivo='', intervalo=5, timeout=300):
    """
    Espera hasta que el archivo del modelo esté disponible en la carpeta especificada.

    :param clase_nombre: Nombre de la clase para formar el nombre de la carpeta.
    :param archivo: Nombre del archivo a buscar (por defecto es 'label_encoder.pkl').
    :param intervalo: Tiempo en segundos entre cada comprobación (por defecto es 5 segundos).
    :param timeout: Tiempo máximo de espera en segundos (por defecto es 300 segundos).
    :return: True si el archivo está disponible dentro del tiempo límite, False si se excede el tiempo de espera.
    """
    ruta = f'./{clase_nombre}/{archivo}'
    tiempo_esperado = 0

    while tiempo_esperado < timeout:
        if os.path.exists(ruta):
            print(f"El archivo '{archivo}' está disponible en la carpeta '{clase_nombre}'.")
            return True
        time.sleep(intervalo)
        tiempo_esperado += intervalo
        print(f"Esperando el archivo '{archivo}' en la carpeta '{clase_nombre}'...")

    print(f"Tiempo de espera agotado. El archivo '{archivo}' no se encontró en la carpeta '{clase_nombre}'.")
    return False

  #-----------------------------------------------------------------------------
  @classmethod
  def usarModelo_RND_Forest(cls,x_predecir):
    # Cargar el modelo
    modelo_cargado = joblib.load(f'./{cls.__name__}/modelo_rnd_forest.pkl')

    # Cargar el codificador de etiquetas
    label_encoder_cargado = joblib.load(f'./{cls.__name__}/label_encoder_rnd_forest.pkl')

    # Realizar predicción con el modelo cargado
    prediccion_nueva = modelo_cargado.predict(x_predecir)
    return label_encoder_cargado.inverse_transform(prediccion_nueva)[0]


## Clase Category1A2

In [ ]:

class Category1A2(Eca):
  params_grid_random_forest=[
      #Probará con 50,100,150 arboles de desición en el randomforest
      {"n_estimators":[50,100,150]},
  ]
  NOMINALS: Dict[str, Union[float, int]] = category1A2Nominals
  PROBES: List[Dict[str, Union[float, int]]] = category1A2Probes
  propertynumber : int = catgory1A2Propertynumber
  HEADERSA: List[Dict[str,str]] = category1A2HeadersA
  HEADERSB: List[Dict[str,str]] = category1A2HeadersB
  HEADERSC: List[Dict[str,str]] = category1A2HeadersC
  HEADERSD: List[Dict[str,str]] = category1A2HeadersD

  #constructor
  def __init__(self,
               o: float,
               dbo: float,
               as_: float,
               cd: float,
               cu: float,
               cr: float,
               fe: float,
               mn: float,
               pb: float,
               hg: float,
               zn: float,
               ph: float,
               coliformes: float):
                super().__init__(_o = o, _dbo = dbo, _as_ = as_,
                                 _cd = cd, _cu = cu, _cr = cr, _fe = fe, _mn = mn, _pb = pb, _hg = hg, _zn = zn, _ph = ph, _coliformes = coliformes)
  #-----------------------------------------------------------------------------
  @classmethod
  def createFromDF(cls,df : pd.DataFrame) -> List['Category1A2'] :
    """
    Crea una lista de instancias de Category1A2 a partir de un DataFrame.

    Itera sobre cada fila del DataFrame y crea una instancia de Category1A2
    para cada una, añadiéndola a una lista.

    Args:
        df (pd.DataFrame): DataFrame que contiene los datos para crear instancias de Category1A2.

    Returns:
        List[Category1A2]: Una lista de objetos Category1A2 creados a partir de los datos del DataFrame.

    Raises:
        KeyError: Si alguna columna esperada falta en el DataFrame.
        Exception: Para cualquier otro error imprevisto durante el procesamiento.
    """
    categories = []
    try:

      for _, row in df.iterrows():

          category = cls(
              o=row['o'], dbo=row['dbo'], as_=row['as'], cd=row['cd'], cu=row['cu'],
              cr=row['cr'], fe=row['fe'], mn=row['mn'], pb=row['pb'], hg=row['hg'],
              zn=row['zn'], ph=row['ph'], coliformes=row['coliformes']
          )
          categories.append(category)
    except KeyError as e:
        print(f"Error de clave: {e}")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    return categories



##Clase Category3

In [ ]:
class Category3(Eca):
  params_grid_random_forest=[
      #Probará con 100,150,200 arboles de desición en el randomforest
      {"n_estimators":[100,150,200]},
  ]
  propertynumber : int = category3Propertynumber
  PROBES: List[Dict[str, Union[float, int]]] = category3Probes

  HEADERSA: List[Dict[str,str]] = category3HeadersA
  HEADERSB: List[Dict[str,str]] = category3HeadersB
  HEADERSC: List[Dict[str,str]] = category3HeadersC
  HEADERSD: List[Dict[str,str]] = category3HeadersD

  #constructor
  @abstractmethod
  def _init_(
      self,
      cloruros: float,
      conductividad: float,
      dbo: float,
      o: float,
      ph: float,
      al: float,
      as_: float,
      b: float,
      cd: float,
      cu: float,
      fe: float,
      mn: float,
      hg: float,
      pb: float,
      zn: float,
      coliformes: float,
      huevos_de_hemintos:float
      ):
        super()._init( _cloruros = cloruros, _conductividad = conductividad, _dbo = dbo, _o = o, _ph = ph, _al = al, _as = as_, _b = b, _cd = cd, _cu = cu, _fe = fe, _mn = mn, _hg = hg, _pb = pb, _zn = zn, _coliformes = coliformes, _huevos_de_hemintos = huevos_de_hemintos,
        )


  #-----------------------------------------------------------------------------

  @classmethod
  def createFromDF(
      cls : Type[Category3Type],
      df : pd.DataFrame) -> List[Category3Type] :

    """
    Crea una lista de instancias de Category1A2 a partir de un DataFrame.

    Itera sobre cada fila del DataFrame y crea una instancia de Category1A2
    para cada una, añadiéndola a una lista.

    Args:
        df (pd.DataFrame): DataFrame que contiene los datos para crear instancias de Category1A2.

    Returns:
        List[Category1A2]: Una lista de objetos Category1A2 creados a partir de los datos del DataFrame.

    Raises:
        KeyError: Si alguna columna esperada falta en el DataFrame.
        Exception: Para cualquier otro error imprevisto durante el procesamiento.
    """
    categories = []
    try:

      for _, row in df.iterrows():

          category = cls(
              cloruros = row['cloruros'],conductividad = row['conductividad'], dbo = row['dbo'], o = row['o'], ph = row['ph'], al = row['al'], as_ = row['as'], b = row['b'], cd = row['cd'], cu = row['cu'],
              fe = row['fe'], mn = row['mn'], hg = row['hg'], pb = row['pb'], zn = row['zn'], coliformes = row['coliformes'], huevos_de_hemintos = row['huevos de hemintos'],
          )

          categories.append(category)

    except KeyError as e:
        print(f"Error de clave: {e}")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    return categories

  #-----------------------------------------------------------------------------

## Clase RestrictedIrrigation



In [ ]:
class RestrictedIrrigation(Category3):

  NOMINALS: Dict[str, Union[float, int]] = restrictedIrrigationNominals

  def __init__(
      self, cloruros: float, conductividad: int, dbo: float, o: float, ph: float, al: float, as_: float, b: float, cd: float, cu: float, fe: float, mn: float, hg: float, pb: float, zn: float, coliformes: float, huevos_de_hemintos: int
      ):
        super().__init__(
            cloruros=cloruros, conductividad=conductividad, dbo=dbo, o=o, ph=ph, al=al,
            as_=as_, b=b, cd=cd, cu=cu, fe=fe, mn=mn, hg=hg, pb=pb, zn=zn, coliformes=coliformes,
            huevos_de_hemintos=huevos_de_hemintos
        )

## Clase NoRestrictedIrrigation

In [ ]:
class NoRestrictedIrrigation(Category3):

  NOMINALS: Dict[str, Union[float, int]] = noRestrictedIrrigationNominals

  def __init__(
      self, cloruros: float, conductividad: int, dbo: float, o: float, ph: float, al: float, as_: float, b: float, cd: float, cu: float, fe: float, mn: float, hg: float, pb: float, zn: float, coliformes: int, huevos_de_hemintos: int
      ):
        super().__init__(
            cloruros=cloruros, conductividad=conductividad, dbo=dbo, o=o, ph=ph, al=al,
            as_=as_, b=b, cd=cd, cu=cu, fe=fe, mn=mn, hg=hg, pb=pb, zn=zn, coliformes=coliformes,
            huevos_de_hemintos=huevos_de_hemintos
        )

## Clase AnimalDrinkingWater

In [ ]:
class AnimalDrinkingWater(Category3):

  NOMINALS: Dict[str, Union[float, int]] = animalDrinkingWaterNominals
  PROBES: List[Dict[str, Union[float, int]]] = animalDrinkingWaterProbes

  def _init_(
      self, conductividad: float, dbo: float, o: float, ph: float, al: float, as_: float, b: float, cd: float, cu: float, mn: float, hg: float, pb: float, zn: float, coliformes: float
      ):
        super()._init_(
            cloruros=0, conductividad=conductividad, dbo=dbo, o=o, ph=ph, al=al,
            as_=as_, b=b, cd=cd, cu=cu, fe=0, mn=mn, hg=hg, pb=pb, zn=zn, coliformes=coliformes,
            huevos_de_hemintos=0
        )

  @classmethod
  def createFromDF(
      cls : Type[Category3Type],
      df : pd.DataFrame) -> List[Category3Type] :

    """
    Crea una lista de instancias de Category1A2 a partir de un DataFrame.

    Itera sobre cada fila del DataFrame y crea una instancia de Category1A2
    para cada una, añadiéndola a una lista.

    Args:
        df (pd.DataFrame): DataFrame que contiene los datos para crear instancias de Category1A2.

    Returns:
        List[Category1A2]: Una lista de objetos Category1A2 creados a partir de los datos del DataFrame.

    Raises:
        KeyError: Si alguna columna esperada falta en el DataFrame.
        Exception: Para cualquier otro error imprevisto durante el procesamiento.
    """
    categories = []
    try:

      for _, row in df.iterrows():

          category = cls(
              conductividad = row['conductividad'], dbo = row['dbo'], o = row['o'], ph = row['ph'], al = row['al'], as_ = row['as'], b = row['b'], cd = row['cd'], cu = row['cu'], mn = row['mn'], hg = row['hg'], pb = row['pb'], zn = row['zn'], coliformes = row['coliformes']
          )

          categories.append(category)

    except KeyError as e:
        print(f"Error de clave: {e}")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    return categories

#Entrenamiento
##SVM


In [ ]:
def train() -> None:
    #Genera SVM
    # Category1A2.generar_SVM_multiclass()### lee directamente de la carpeta Category1A2
    # RestrictedIrrigation.generar_SVM_multiclass()### lee directamente de la carpeta RestrictedIrrigation
    # NoRestrictedIrrigation.generar_SVM_multiclass()### lee directamente de la carpeta NoRestrictedIrrigation
    AnimalDrinkingWater.generar_SVM_multiclass()### lee directamente de la carpeta AnimalDrinkingWater

    #Genera random forest
    # Category1A2.generar_randomForest_multiclass()### lee directamente de la carpeta Category1A2
    # RestrictedIrrigation.generar_randomForest_multiclass()### lee directamente de la carpeta RestrictedIrrigation
    # NoRestrictedIrrigation.generar_randomForest_multiclass()### lee directamente de la carpeta NoRestrictedIrrigation
    AnimalDrinkingWater.generar_randomForest_multiclass()### lee directamente de la carpeta AnimalDrinkingWater


# Algoritmos

##Categoría 1A2

In [ ]:
def ica_Category1A2() -> None:

  print("\n\n\n")

  #Se lee el archivo template
  reads = Category1A2.createFromExcel()
  if(len(reads) == 0): #Si hay un error retorna
    return

  #Cálculo del ica
  ica = Category1A2.calculateIca(reads)
  labelAsigned = Category1A2.asignLabel(ica)

  f1,f2,f3=Category1A2.calculateF1(reads),Category1A2.calculateF2(reads),Category1A2.calculateF3(reads)
  x=[[f1,f2,f3]]

  svm = Category1A2.usarModeloSVM(x)##usa el modelo SVM

  rndmforest = Category1A2.usarModelo_RND_Forest(x)
  knn = Category1A2.clasifyKNN([[f1],[f2],[f3]])

  titles : Dict[str,str] = {
      'category' : 'Categoria 1A2',
      'ica' : str(round(ica,2)),
      'formula' : labelAsigned,
      'svm' : svm,
      'rndmforest' : rndmforest,
      'knn' : knn
  }
  Category1A2.plotCollection(reads,titles)

  ##----------------Revisar manejo de errores de svm ------------------

  # Category1A2.generar_SVM_multiclass()### lee directamente de la carpeta Category1A2_excels


  # Category1A2.usarModeloSVM(x)##usa el modelo SVM

  #Predecir KNN


## Categoría 3

### Riego Restringido

In [ ]:
def ica_RestrictedIrrigation() -> None :

  print("\n")
  #Se lee el archivo template
  reads = RestrictedIrrigation.createFromExcel()
  if(len(reads) == 0): #Si hay un error retorna
    return

  #--------------------ICA-----------------------
  ica = RestrictedIrrigation.calculateIca(reads)
  labelAsigned = RestrictedIrrigation.asignLabel(ica)

  ##----------------SVM------------------
  f1,f2,f3=RestrictedIrrigation.calculateF1(reads),RestrictedIrrigation.calculateF2(reads),RestrictedIrrigation.calculateF3(reads)
  x=[[f1,f2,f3]]
  svm = RestrictedIrrigation.usarModeloSVM(x)##usa el modelo SVM
  rndmforest = RestrictedIrrigation.usarModelo_RND_Forest(x)##usa el modelo randomforest
  ##-----------------Plot-----------------
  titles : Dict[str,str] = {
      'category' : 'Categoria 3 Riego Restringido',
      'ica' : str(round(ica,2)),
      'formula' : labelAsigned,
      'svm' : svm,
      'rndmforest':rndmforest
  }
  RestrictedIrrigation.plotCollection(reads,titles)

### Riego No Restringido

In [ ]:
def ica_NoRestrictedIrrigation() -> None :

  print("\n")
  #Se lee el archivo template
  reads = NoRestrictedIrrigation.createFromExcel()
  if(len(reads) == 0): #Si hay un error retorna
    return

  ##----------------ICA-----------------------
  ica = NoRestrictedIrrigation.calculateIca(reads)
  labelAsigned = NoRestrictedIrrigation.asignLabel(ica)

  ##----------------SVM------------------
  f1,f2,f3=NoRestrictedIrrigation.calculateF1(reads),NoRestrictedIrrigation.calculateF2(reads),NoRestrictedIrrigation.calculateF3(reads)
  x=[[f1,f2,f3]]
  svm =  NoRestrictedIrrigation.usarModeloSVM(x)##usa el modelo SVM
  rndmforest = NoRestrictedIrrigation.usarModelo_RND_Forest(x)##usa el modelo randomforest
  ##----------------Plot---------------------------------
  titles : Dict[str,str] = {
      'category' : 'Categoria 3 Riego No Restringido',
      'ica' : str(round(ica,2)),
      'formula' : labelAsigned,
      'svm' : svm,
      'rndmforest':rndmforest

  }

  NoRestrictedIrrigation.plotCollection(reads,titles)

### Bebida de animales

In [ ]:
def ica_AnimalDrinkingWater() -> None :

  print("\n")
  #Se lee el archivo template
  reads = AnimalDrinkingWater.createFromExcel()
  if(len(reads) == 0): #Si hay un error retorna
    return

  ##----------------ICA------------------------
  ica = AnimalDrinkingWater.calculateIca(reads)
  labelAsigned = AnimalDrinkingWater.asignLabel(ica)

  ##----------------SVM------------------
  f1,f2,f3=AnimalDrinkingWater.calculateF1(reads),AnimalDrinkingWater.calculateF2(reads),AnimalDrinkingWater.calculateF3(reads)
  x=[[f1,f2,f3]]
  svm = AnimalDrinkingWater.usarModeloSVM(x)##usa el modelo SVM
  rndmforest = AnimalDrinkingWater.usarModelo_RND_Forest(x)##usa el modelo randomforest
  #-----------------Plot------------------
  titles : Dict[str,str] = {
      'category' : 'Categoria 3 Bebida de animales',
      'ica' : str(round(ica,2)),
      'formula' : labelAsigned,
      'svm' : svm,
      'rndmforest':rndmforest

  }
  AnimalDrinkingWater.plotCollection(reads,titles)

## Función Principal


In [ ]:
def __CALCULATE_ICA__():

  train()

  # ica_Category1A2()

  ica_AnimalDrinkingWater()

__CALCULATE_ICA__()


/content/./entrenamiento/randomDf_final_train_categoria_bebida_animal.xlsx ----
/content/./entrenamiento/randomDf_final_train_categoria_bebida_animal.xlsx
Ocurrió un error: Can't instantiate abstract class AnimalDrinkingWater with abstract method __init__


ZeroDivisionError: division by zero

###Hasta aquí codigo chido

In [ ]:
###SVM(Suport Vector Machine)
##Multi-class classification

In [ ]:
import matplotlib.pyplot as plt

from sklearn import datasets, svm
from sklearn.inspection import DecisionBoundaryDisplay

# import some data to play with
# iris = datasets.load_iris()
# Take the first two features. We could avoid this by using a two-dim dataset
X = factoresefe
Y = labels

# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
models = (
    svm.SVC(kernel="linear", C=C),
    svm.LinearSVC(C=C, max_iter=10000),
    svm.SVC(kernel="rbf", gamma=0.7, C=C),
    svm.SVC(kernel="poly", degree=3, gamma="auto", C=C),
    svm.SVC(decision_function_shape='ovo')

)
models = (clf.fit(X, y) for clf in models)

# title for the plots
titles = (
    "SVC with linear kernel",
    "LinearSVC (linear kernel)",
    "SVC with RBF kernel",
    "SVC with polynomial (degree 3) kernel",
)

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = X[0], X[1]

for clf, title, ax in zip(models, titles, sub.flatten()):
    disp = DecisionBoundaryDisplay.from_estimator(
        clf,
        X,
        response_method="predict",
        cmap=plt.cm.coolwarm,
        alpha=0.8,
        ax=ax,
        # xlabel=iris.feature_names[0],
        # ylabel=iris.feature_names[1],
    )
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

plt.show()

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

# Datos de entrada (características)
X = [[0.5384615384615384, 0.34615384615384615, 71.50487223318459],
     [0.5384615384615384, 0.1958041958041958, 97.5080952054009],
     [0.07692307692307693, 0.019230769230769232, 10.318913190095385]]

# Etiquetas de clase
labels = ['Regular', 'Malo', 'Excelente']

# Codificar etiquetas de clase
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(labels)

# Crear un clasificador SVM con la estrategia "one-vs-one"
clf = svm.SVC(decision_function_shape='ovo')

# Entrenar el modelo
clf.fit(X, Y)

# Datos de prueba
X_test = [[0.3, 0.1, 50]]  # Ejemplo de características para un nuevo dato

# Realizar predicción
prediccion = clf.predict(X_test)
print("Predicción:", label_encoder.inverse_transform(prediccion))  # Decodificar predicción

# Obtener la función de decisión
dec = clf.decision_function(X_test)
print("Valores de la función de decisión:", dec)

# Verificar la forma de la salida de la función de decisión
print("Forma de la salida de la función de decisión:", dec.shape)

# Cambiar la estrategia a "one-vs-rest"
clf.decision_function_shape = "ovr"

# Obtener la función de decisión con la nueva estrategia
dec = clf.decision_function(X_test)
print("Valores de la función de decisión con 'ovr':", dec)
print("Forma de la salida de la función de decisión con 'ovr':", dec.shape)


In [ ]:
clf.decision_function_shape = "ovr"
dec = clf.decision_function([factoresefe[0]])
dec.shape[1] # 4 classes

###Leer los excels originales

In [ ]:
categoria1=[]
categoria2=[]
categoria3=[]
contador1=0
contador2=0
contador3=0
archivosDict=[
    {"ruta":"/content/./exceles_carpeta/Monetoreo_de_parametros_cuenca_Huarmey_2023_data.xlsx","especificaciones":{
        "hojas": {
            "c1":{"inicio":"B:P","skiprows":8,"categoría":1},
            "c2":{"inicio":"B:J","skiprows":8,"categoría":2}
            },
        "nuevosTitulos":{
            "c1":["Parametro",
                  "Unidades",
                  "ECA - Agua, D.S N°004-2017 - MINAM. Categoría 1: Poblacional y recreacional. Subcategoría A: Aguas superficiales destinadas a la producción de agua. A2: Aguas que pueden ser potabilizadas con tratamiento convencional.",
                  "QHerc1","QHerc2","QHerc3","QSant1","QYact1","Agua para riego no","Agua parariego restringido","D2: Bebida de Animales","QMont2","QPall3","RAija1","Datos Resaltados"],
            "c2":["Parametro",
                  "Unidades",
                  "ECA - Agua, D.S N°004-2017 - MINAM. Categoría 1: Poblacional y recreacional. Subcategoría B: Aguas superficiales destinadas para recreación. B1: Contacto Primario.",
                  "MHuar2S","MHuar3S","MHuar4S","MHuar5","MHuar7","Datos Resaltados"]
            },
        }},
    {"ruta":"/content/./exceles_carpeta/Montoreo_de_parametros_de_agua_cuenca_Huarmey_2020_data.xlsx","especificaciones":{
        "hojas": {
            "Eca3":{"inicio":"B:P","skiprows":6,"categoría":3},
            "Eca3M":{"inicio":"B:M","skiprows":6,"categoría":3},
            "Eca1":{"inicio":"B:I","skiprows":6,"categoría":1}
            },
        "nuevosTitulos":{
            "Eca3":["Parametro","Unidades","Riesgo no restringido","Riesgo restringido","D2:Bebida de animales","QMont1","QMont2","QMont3","QHuin1","RLlac1","QHerc3","QPall1","QSant1","QSant2","Datos Resaltados"],
            "Eca3M":["Parametro","Unidades","Riesgo no restringido","Riesgo restringido","D2:Bebida de animales","Rsant1","RAija1","RCota3","RCota2","RAija4","RAija3","DatosResaltados"],
            "Eca1":["Parametro","Unidades","ECA - AGUA CATEG.1-A2","RLame1","RLame2","RMall1","RMall2","Datos Resaltados"]
            },        }},
    {"ruta":"/content/./exceles_carpeta/Monetoreo_de_parametros_cuenca_Huarmey_2021_data.xlsx","especificaciones":{
        "hojas": {
            "eca1":{"inicio":"B:P","skiprows":12,"categoría":1},
            "eca1.2":{"inicio":"B:P","skiprows":12,"categoría":1},
            "eca2":{"inicio":"B:V","skiprows":12,"categoría":2},
            "eca2.3":{"inicio":"B:V","skiprows":12,"categoría":2}
            },
        "nuevosTitulos":{
            "eca1":["Parametro","Unidades","Cat.1-A2","QHero3","QHuin1","QMacs2","QNnom2","QSant1","QSant4","QYact1","RLmer1","RLmer2","RSant1","RYact1","Datos Resaltados"],
            "eca1.2":["Parametro","Unidades","Cat.1-A2","QHero3","QHuin1","QMacs2","QNnom2","QSant1","QSant4","QYact1","RLmer1","RLmer2","RSant1","RYact1","Datos Resaltados"],
            "eca2":["Parametro","Unidades","Cat.3-D1","Cat.3-D2","QMont1","QMont2","QMont3","QPall3","RAija1","RAija2","RAija3","RAija4","RCota1","RCota2","RCota3","RHuar1","RHuar3","RMall1","RMalv1","RMalv2","Datos Resaltados"],
            "eca2.3":["Parametro","Unidades","Cat.3-D1","Cat.3-D2","QMont1","QMont2","QMont3","QPall3","RAija1","RAija2","RAija3","RAija4","RCota1","RCota2","RCota3","RHuar1","RHuar3","RMall1","RMalv1","RMalv2","Datos Resaltados"]
            },
        }}
]
for archivo in archivosDict:
  exceltemp=pd.ExcelFile(archivo["ruta"])
  for i in exceltemp.sheet_names:
    skiprowtemp=archivo["especificaciones"]["hojas"][str(i)]["skiprows"]
    usecolstemp=archivo["especificaciones"]["hojas"][str(i)]["inicio"]
    categoriatemp=archivo["especificaciones"]["hojas"][str(i)]["categoría"]
    dftemp = pd.read_excel(archivo["ruta"], sheet_name=i,skiprows=skiprowtemp,usecols=usecolstemp, header=None)
    dftemp.columns = archivo["especificaciones"]["nuevosTitulos"][str(i)]
    dftemp=dftemp.transpose()
    # Detecta filas y columnas vacías
    if categoriatemp==1:
      categoria1.append(dftemp)
      print(f'posición {contador1} en el array de categoria1 ---> {archivo["ruta"]}, hoja: {i}')
      contador1+=1
    elif categoriatemp==2:
      categoria2.append(dftemp)
      print(f'posición {contador2} en el array de categoria2 ---> {archivo["ruta"]}, hoja: {i}')
      contador2+=1
    else:
      categoria3.append(dftemp)
      print(f'posición {contador3} en el array de categoria3 ---> {archivo["ruta"]}, hoja: {i}')
      contador3+=1


In [ ]:
#Aqui tratamiento para cada tabla para poder obtener correctamente los datos... aquí me quedé
for dftemp in categoria1:
  display(dftemp)


##SVM (Support Vector Machine)

In [ ]:
# aca le chambeo yo

###RANDOM FOREST

In [ ]:
# Modelo entrenado con el conjunto de datos sin escalar
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from sklearn.preprocessing import LabelEncoder



X=[[0.5384615384615384, 0.34615384615384615, 71.50487223318459], [0.5384615384615384, 0.1958041958041958, 97.5080952054009], [0.07692307692307693, 0.019230769230769232, 10.318913190095385]]
y=['Regular', 'Malo', 'Excelente']
# Codificar las etiquetas de texto a valores numéricos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Entrenar el clasificador RandomForest
clf_rnd = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf_rnd.fit(X, y)

Z = clf_rnd.predict([X[0]])
print(Z)



In [ ]:
y_train_pred = clf_rnd.predict(X)
print(y_train_pred)

In [ ]:
print("F1 Score Train Set:", f1_score(y_train_pred, y, average='weighted'))

In [ ]:
n_classes = 3
n_estimators = 30
cmap = plt.cm.RdYlBu
plot_step = 0.02  # fine step width for decision surface contours
plot_step_coarser = 0.5  # step widths for coarse classifier guesses
RANDOM_SEED = 13  # fix the seed on each iteration

# Load data
iris = load_iris()

plot_idx = 1

models = [
    DecisionTreeClassifier(max_depth=None),
    RandomForestClassifier(n_estimators=n_estimators),
    ExtraTreesClassifier(n_estimators=n_estimators),
    AdaBoostClassifier(
        DecisionTreeClassifier(max_depth=3),
        n_estimators=n_estimators,
        algorithm="SAMME",
    ),
]

for pair in ([0, 1], [0, 2], [2, 3]):
    for model in models:
        # We only take the two corresponding features
        X = iris.data[:, pair]
        y = iris.target

        # Shuffle
        idx = np.arange(X.shape[0])
        np.random.seed(RANDOM_SEED)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        # Standardize
        mean = X.mean(axis=0)
        std = X.std(axis=0)
        X = (X - mean) / std

        # Train
        model.fit(X, y)

        scores = model.score(X, y)
        # Create a title for each column and the console by using str() and
        # slicing away useless parts of the string
        model_title = str(type(model)).split(".")[-1][:-2][: -len("Classifier")]

        model_details = model_title
        if hasattr(model, "estimators_"):
            model_details += " with {} estimators".format(len(model.estimators_))
        print(model_details + " with features", pair, "has a score of", scores)

        plt.subplot(3, 4, plot_idx)
        if plot_idx <= len(models):
            # Add a title at the top of each column
            plt.title(model_title, fontsize=9)

        # Now plot the decision boundary using a fine mesh as input to a
        # filled contour plot
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(
            np.arange(x_min, x_max, plot_step), np.arange(y_min, y_max, plot_step)
        )

        # Plot either a single DecisionTreeClassifier or alpha blend the
        # decision surfaces of the ensemble of classifiers
        if isinstance(model, DecisionTreeClassifier):
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            cs = plt.contourf(xx, yy, Z, cmap=cmap)
        else:
            # Choose alpha blend level with respect to the number
            # of estimators
            # that are in use (noting that AdaBoost can use fewer estimators
            # than its maximum if it achieves a good enough fit early on)
            estimator_alpha = 1.0 / len(model.estimators_)
            for tree in model.estimators_:
                Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
                Z = Z.reshape(xx.shape)
                cs = plt.contourf(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)

        # Build a coarser grid to plot a set of ensemble classifications
        # to show how these are different to what we see in the decision
        # surfaces. These points are regularly space and do not have a
        # black outline
        xx_coarser, yy_coarser = np.meshgrid(
            np.arange(x_min, x_max, plot_step_coarser),
            np.arange(y_min, y_max, plot_step_coarser),
        )
        Z_points_coarser = model.predict(
            np.c_[xx_coarser.ravel(), yy_coarser.ravel()]
        ).reshape(xx_coarser.shape)
        cs_points = plt.scatter(
            xx_coarser,
            yy_coarser,
            s=15,
            c=Z_points_coarser,
            cmap=cmap,
            edgecolors="none",
        )

        # Plot the training points, these are clustered together and have a
        # black outline
        plt.scatter(
            X[:, 0],
            X[:, 1],
            c=y,
            cmap=ListedColormap(["r", "y", "b"]),
            edgecolor="k",
            s=20,
        )
        plot_idx += 1  # move on to the next plot in sequence

plt.suptitle("Classifiers on feature subsets of the Iris dataset", fontsize=12)
plt.axis("tight")
plt.tight_layout(h_pad=0.2, w_pad=0.2, pad=2.5)
plt.show()

###Generación de datos

In [ ]:
import pandas as pd
#https://www.kaggle.com/datasets/mssmartypants/water-quality
df = pd.read_csv('./waterQuality1.csv')
df = df[(df != 0).all(axis=1)]

df['arsenic'] = df['arsenic']
df['cadmium'] = df['cadmium']
df['copper'] = df['copper'] *3
df['chromium'] = df['chromium']*10
df['lead'] = df['lead']*2
df['mercury'] = df['mercury'] *100

df.head()
df_subset = df.head(400)

# Reiniciar el índice del DataFrame seleccionado
df_sample = df_subset.reset_index(drop=True)
df_sample.drop('radium', axis=1, inplace=True)
df_sample.drop('is_safe', axis=1, inplace=True)
df_sample.drop('uranium', axis=1, inplace=True)
df_sample.drop('silver', axis=1, inplace=True)
df_sample.drop('selenium', axis=1, inplace=True)
df_sample.drop('perchlorate', axis=1, inplace=True)
df_sample.drop('flouride', axis=1, inplace=True)
df_sample.drop('nitrites', axis=1, inplace=True)
df_sample.drop('bacteria', axis=1, inplace=True)
df_sample.drop('viruses', axis=1, inplace=True)
df_sample.drop('nitrates', axis=1, inplace=True)
df_sample.drop('ammonia', axis=1, inplace=True)
df_sample.drop('chloramine', axis=1, inplace=True)
df_sample = df_sample.rename(columns={'aluminium': 'al'})
df_sample = df_sample.rename(columns={'arsenic': 'as'})
df_sample = df_sample.rename(columns={'barium': 'b'})
df_sample = df_sample.rename(columns={'cadmium': 'cd'})
df_sample = df_sample.rename(columns={'chromium': 'cr'})
df_sample = df_sample.rename(columns={'copper': 'cu'})
df_sample = df_sample.rename(columns={'lead': 'pb'})
df_sample = df_sample.rename(columns={'mercury': 'hg'})
df_sample.info()


In [ ]:
df = pd.read_csv('./water_potability.csv')
df = df[(df != 0).all(axis=1)]

# Tomar los primeros 2000 datos
df_subset = df.head(400)

# Reiniciar el índice del DataFrame seleccionado
df = df_subset.reset_index(drop=True)
df.drop('Hardness', axis=1, inplace=True)
df.drop('Solids', axis=1, inplace=True)
df.drop('Chloramines', axis=1, inplace=True)
df.drop('Sulfate', axis=1, inplace=True)
df.drop('Organic_carbon', axis=1, inplace=True)
df.drop('Turbidity', axis=1, inplace=True)
df.drop('Trihalomethanes', axis=1, inplace=True)
df.drop('Potability', axis=1, inplace=True)
df = df.rename(columns={'Conductivity': 'conductividad'})

df_concat = pd.concat([df, df_sample], axis=1)

df_concat.head()
# df_concat.to_excel('data_incompleta.xlsx', index=False)

In [ ]:
df_concat.info()

In [ ]:
columnas=["al","as","b","cd","cr","cu","dbo","fe","hg","mn","o","pb","zn","ph","conductividad","coliformes"]
import numpy as np
for columna in columnas:
  if columna in df_concat.columns:
      df_concat[columna] = df_concat[columna].fillna(df_concat[columna].max())
    # df['Sulfate'] = df['Sulfate'].fillna(df['Sulfate'].median())
    # df['Trihalomethanes'] = df['Trihalomethanes'].fillna(df['Trihalomethanes'].median())
columnasNoHay=['dbo', 'fe', 'mn', 'o', 'zn', 'coliformes']
rangos=[[.01,10],[.001,5],[.01,10],[0.2,20],[00.1,10],[500,3000]]
for columna,rango in zip(columnasNoHay,rangos):
  df_concat[columna] = np.random.uniform(rango[0], rango[1], len(df_concat))

df_concat.info()

In [ ]:
df_concat_ordenado=df_concat[columnas]
df_mini=df_concat_ordenado.head(19)


#https://www.kaggle.com/code/paviah/water-quality-prediction-mohpo-with-xgbclassifier
import io

display(df_mini)


In [ ]:
import numpy as np

def guardar_df_en_hojas_aleatorias(df, nombre_archivo):
    # Contador para las hojas
    hoja_num = 1
    with pd.ExcelWriter(nombre_archivo) as writer:
      # while not df.empty:
      #     # Elegir aleatoriamente el número de filas para la siguiente hoja (entre 4 y 12)
      #     num_filas = np.random.randint(4, 13)

      #     # Seleccionar las primeras 'num_filas' filas del DataFrame
      #     df_hoja = df.iloc[:num_filas]

      #     # Guardar esas filas en una hoja nueva
      #     # df_hoja.to_excel(writer, sheet_name=f'Hoja{hoja_num}', index=False)
          df.to_excel(writer, sheet_name=f'Hoja1', index=False)

          # # Eliminar esas filas del DataFrame original
          # df = df.iloc[num_filas:]

          # # Incrementar el contador de hojas
          # hoja_num += 1
contains_nan = df_concat.isna().values.any()

print("¿Contiene NaN?", contains_nan)
df_concat.to_csv('archivo_aleatorio.csv', index=False)
guardar_df_en_hojas_aleatorias(df_concat, 'archivo_aleatorio.xlsx')

In [ ]:
df_concat.iloc[1009:1013]

In [ ]:
(df_concat["hg"].max())
(df_concat["cu"].max())
(df_concat["as"].max())
(df_concat["cd"].max())

### Exagerados para pesimo


In [ ]:
import pandas as pd
#https://www.kaggle.com/datasets/mssmartypants/water-quality
df = pd.read_csv('./waterQuality1.csv')
df = df[(df != 0).all(axis=1)]

df['arsenic'] = df['arsenic']*1000
df['cadmium'] = df['cadmium']*1000
df['copper'] = df['copper'] *1000
df['chromium'] = df['chromium']*1000
df['lead'] = df['lead']*1000
df['mercury'] = df['mercury'] *1000

df.head()
df_subset = df.head(4000)

# Reiniciar el índice del DataFrame seleccionado
df_sample = df_subset.reset_index(drop=True)
df_sample.drop('radium', axis=1, inplace=True)
df_sample.drop('is_safe', axis=1, inplace=True)
df_sample.drop('uranium', axis=1, inplace=True)
df_sample.drop('silver', axis=1, inplace=True)
df_sample.drop('selenium', axis=1, inplace=True)
df_sample.drop('perchlorate', axis=1, inplace=True)
df_sample.drop('flouride', axis=1, inplace=True)
df_sample.drop('nitrites', axis=1, inplace=True)
df_sample.drop('bacteria', axis=1, inplace=True)
df_sample.drop('viruses', axis=1, inplace=True)
df_sample.drop('nitrates', axis=1, inplace=True)
df_sample.drop('ammonia', axis=1, inplace=True)
df_sample.drop('chloramine', axis=1, inplace=True)
df_sample = df_sample.rename(columns={'aluminium': 'al'})
df_sample = df_sample.rename(columns={'arsenic': 'as'})
df_sample = df_sample.rename(columns={'barium': 'b'})
df_sample = df_sample.rename(columns={'cadmium': 'cd'})
df_sample = df_sample.rename(columns={'chromium': 'cr'})
df_sample = df_sample.rename(columns={'copper': 'cu'})
df_sample = df_sample.rename(columns={'lead': 'pb'})
df_sample = df_sample.rename(columns={'mercury': 'hg'})
df_sample.info()


In [ ]:
df = pd.read_csv('./water_potability.csv')
df = df[(df != 0).all(axis=1)]

# Tomar los primeros 2000 datos
df_filtrado = df[(df['ph'] < 5.5) | (df['ph'] > 9)]

df_subset = df_filtrado.head(4000)

# Reiniciar el índice del DataFrame seleccionado
df = df_subset.reset_index(drop=True)
df.drop('Hardness', axis=1, inplace=True)
df.drop('Solids', axis=1, inplace=True)
df.drop('Chloramines', axis=1, inplace=True)
df.drop('Sulfate', axis=1, inplace=True)
df.drop('Organic_carbon', axis=1, inplace=True)
df.drop('Turbidity', axis=1, inplace=True)
df.drop('Trihalomethanes', axis=1, inplace=True)
df.drop('Potability', axis=1, inplace=True)
df = df.rename(columns={'Conductivity': 'conductividad'})

df_concat = pd.concat([df, df_sample], axis=1)

df_concat.head()
# df_concat.to_excel('data_incompleta.xlsx', index=False)

In [ ]:
columnas=["al","as","b","cd","cr","cu","dbo","fe","hg","mn","o","pb","zn","ph","conductividad","coliformes"]
import numpy as np
for columna in columnas:
  if columna in df_concat.columns:
      df_concat[columna] = df_concat[columna].fillna(df_concat[columna].max())
    # df['Sulfate'] = df['Sulfate'].fillna(df['Sulfate'].median())
    # df['Trihalomethanes'] = df['Trihalomethanes'].fillna(df['Trihalomethanes'].median())
columnasNoHay=['dbo', 'fe', 'mn', 'o', 'zn', 'coliformes']
rangos=[[1,1000],[1,1000],[1,1000],[1,1000],[1,100],[2000,100000]]
for columna,rango in zip(columnasNoHay,rangos):
  df_concat[columna] = np.random.uniform(rango[0], rango[1], len(df_concat))

df_concat.info()

In [ ]:
df_concat_ordenado=df_concat[columnas]
df_mini=df_concat_ordenado.head(19)


#https://www.kaggle.com/code/paviah/water-quality-prediction-mohpo-with-xgbclassifier
import io

display(df_mini)


In [ ]:
import numpy as np

def guardar_df_en_hojas_aleatorias(df, nombre_archivo):
    # Contador para las hojas
    hoja_num = 1
    with pd.ExcelWriter(nombre_archivo) as writer:
      # while not df.empty:
      #     # Elegir aleatoriamente el número de filas para la siguiente hoja (entre 4 y 12)
      #     num_filas = np.random.randint(4, 13)

      #     # Seleccionar las primeras 'num_filas' filas del DataFrame
      #     df_hoja = df.iloc[:num_filas]

      #     # Guardar esas filas en una hoja nueva
      #     # df_hoja.to_excel(writer, sheet_name=f'Hoja{hoja_num}', index=False)
          df.to_excel(writer, sheet_name=f'Hoja1', index=False)

          # # Eliminar esas filas del DataFrame original
          # df = df.iloc[num_filas:]

          # # Incrementar el contador de hojas
          # hoja_num += 1
contains_nan = df_concat.isna().values.any()

print("¿Contiene NaN?", contains_nan)
guardar_df_en_hojas_aleatorias(df_concat, 'archivo_aleatorio_muy_exageraado.xlsx')

In [ ]:
df_concat.iloc[1009:1013]

In [ ]:
(df_concat["hg"].max())
(df_concat["cu"].max())
(df_concat["as"].max())
(df_concat["cd"].max())

In [ ]:
@classmethod
def read_doc(cls, carpeta : str = './template' ) -> pd.DataFrame:
    """
    Lee datos de un archivo de Excel ubicado en una carpeta específica.

    Define la ruta del archivo de Excel y, si el archivo existe, lo lee utilizando `pd.read_excel`
    y devuelve el contenido como un DataFrame. Si el archivo no se encuentra, imprime un mensaje de error.

    Returns:
        pd.DataFrame: Los datos del archivo de Excel leídos como un DataFrame.

    Prints:
        str: Mensaje de error si el archivo no se encuentra en la carpeta.
    """

    #Se define el dorectorio para guardar el template
    folder_path = f'/content/{carpeta}'

    if not os.path.exists(carpeta):
      os.makedirs(f'{carpeta}')
      print(f"Directorio '{f'{carpeta}'}' creado.")
      print(f'sube tus archivo de entrenamiento a la carpeta {carpeta}')
      return  pd.DataFrame()
    archivosEncarpeta = [
      os.path.join(os.getcwd(), f'{carpeta}', x)
      for x in os.listdir(f'{carpeta}')
      if not x.startswith(".ipynb_checkpoints")  # Filtrar el archivo .ipynb_checkpoints
      ] # genera todas las rutas de los xslx para leerlos
    # print(f'La carpeta POS contiene : {str(len(categoria1paths))} archivos')
    if len(archivosEncarpeta) ==0:
      print(f"Capeta {carpeta} vacía sube tu excel")
      return  pd.DataFrame()
    #Intenta leer el archivo
    full_path = os.path.join(folder_path, archivosEncarpeta[0])
    if os.path.exists(full_path):
        data = pd.read_excel(full_path, engine='openpyxl')
        return data

    print(f'No se ha encontrado el archivo en la carpeta {carpeta}')
    return  pd.DataFrame()
